In [1]:
import re
import os

In [2]:
pathFileOrigemExtraido = 'DOWNLOAD/EXTRAIDO/'
pathFileDestinoProcessado = 'DOWNLOAD/PROCESSADO/'

In [3]:
textoOriginal = []
txtNew = []
listSqlIncra = []
caderno = ''
data = ''
nmArqOriginal = ''

In [4]:
def preProcessarTextoRemoverIfen():
    
    textoOriginal.reverse()
    
    for linha in textoOriginal:

        idx = textoOriginal.index(linha)

        if linha[-1:] == '-' :

            idxAux = idx - 1
            newTxt = textoOriginal[idx][:-1] + textoOriginal[idxAux] 
            txtNew.pop()
            txtNew.append(newTxt)

            textoOriginal[idx] = newTxt

        else:
            txtNew.append(textoOriginal[idx])
    
    txtNew.reverse()

In [5]:
def separarInformacoesSQLIncra():
    
    bagOfWordsTitulo = ['RECONSIDERACAO','ALVARA','REVALIDACAO','CERTIFICADO']
    
    for linha in txtNew:
    
        if(re.findall(r'^\d{4}\-\d{7}\-\d{1}\s[SQL\/INCRA]', linha)):

            protocolo = ''
            nrSqlIncra = ''
            digito = ''
            nome = ''
            titulo = ''
            status = ''
            #descricao = ''

            idx = txtNew.index(linha)
            
            for x in range(1,5):

                if(re.findall(r'^(INDEFERIDO|DEFERIDO)[\:]$',txtNew[idx + x])):

                    status = txtNew[idx + x]
                    #descricao =  ' '.join(txtNew[idx + x + 1 : idx + x + 7 ] if status == 'INDEFERIDO' else txtNew[ idx + x +1 :idx + x + 4])

                    if x == 2: # sem ajuste

                        nome = txtNew[idx] # linha
                        titulo = txtNew[idx + 1] # titulo

                    elif x == 3:

                        flgIsTitulo = True

                        for palavra in bagOfWordsTitulo:
                            if txtNew[idx + 2].find(palavra) > -1:
                                flgIsTitulo = False
                                break

                        if flgIsTitulo: # alterar titulo
                            nome = txtNew[idx] # linha
                            titulo = txtNew[idx + 1] + txtNew[idx + 2] # titulo
                        else: # alterar nome
                            nome = txtNew[idx] + txtNew[idx + 1] # linha
                            titulo = txtNew[idx + 2] # titulo

                    elif x == 4: # ajustar titulo e razao social
                        nome = txtNew[idx] + ' ' + txtNew[idx + 1]#linha
                        titulo = txtNew[idx + 2] + ' ' + txtNew[idx + 3]#titulo

                    break

            try:
                protocolo = re.findall(r'^\d{4}\-\d{7}\-\d{1}', nome.replace(" ", ""))[0]
                nrSqlIncra = re.findall(r'\d{13}\-.{1}', nome.replace(" ", ""))[0]
                digito = re.findall(r'\s(\d{1}|\d{2}|\d{3})\s', nome)[-1]
            except:
                print 'This is an error message!'
            
            newLine = data + ';' + caderno + ';' + protocolo + ';' + nrSqlIncra + ';' + digito + ';' + nome[45:].strip() + ';' + titulo + ';' + status[:-1]# + ';' + descricao
            novoArquivo('sqlincra', newLine)
            

In [6]:
def separarInformacoesParaProximasConvocacoes():
    for linha in txtNew:
        if(re.findall(r'^\d{4}\-\d{7}\-\d{1}\d{13}\-\d{1}(\d{1}|\d{2}|\d{3})', linha.replace(" ",""))):
            protocolo = re.findall(r'^\d{4}\-\d{7}\-\d{1}', linha.replace(" ",""))[0]
            nrSqlIncra = re.findall(r'\d{13}\-\d{1}', linha.replace(" ",""))[0]
            digito = re.findall(r'\s(\d{1}|\d{2}|\d{3})\s', linha)[-1]
            nome = ' '.join(linha.split(' ')[3:])
            
            newLine =  data + ';' + caderno + ';' +linha + ';' + protocolo + ';' + nrSqlIncra + ';' + digito + ';' + nome
            novoArquivo('proxConvocacoes', newLine)

In [7]:
def novoArquivo(tipo, newLine):
    
    f = open(pathFileDestinoProcessado + nmArquivoOriginal + '_' + tipo + ".csv","a+")
    f.write(newLine + '\n')
    f.close()
    

In [9]:
caminho = [os.path.join(pathFileOrigemExtraido, nome) for nome in os.listdir(pathFileOrigemExtraido)]
arquivos = [arq for arq in caminho if os.path.isfile(arq)]

for arq in arquivos:
    
    print os.path.basename(arq)
    
    nmArquivoOriginal = os.path.basename(arq)[:-4]
    
    caderno = nmArquivoOriginal.split('_')[-1]
    data = '_'.join(nmArquivoOriginal.split('_')[:3])
    
    textoOriginal = []
    
    with open(arq, 'r') as fileopen:
        
        for line in fileopen:
            if not len(line.strip()) < 1 :
                textoOriginal.append(line.strip().upper())
        
        preProcessarTextoRemoverIfen()
        separarInformacoesSQLIncra()
        separarInformacoesParaProximasConvocacoes()

2018_Dezembro_14_cidade.txt
This is an error message!
This is an error message!
2018_Dezembro_15_cidade.txt
This is an error message!
This is an error message!
